In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from pprint import pprint
import json
from datasets import load_from_disk
from transformers import RobertaTokenizer
from process_java_model import *


In [3]:
dataset = load_from_disk("/data/nicolasmaier/dataset/hf_clean_dataset")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['code', 'contents', 'xmi', 'originalLine', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 425631
    })
    valid: Dataset({
        features: ['code', 'contents', 'xmi', 'originalLine', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14634
    })
    test: Dataset({
        features: ['code', 'contents', 'xmi', 'originalLine', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 25156
    })
})


In [8]:
offset = 103000
n = 10000

for i in range(offset, offset + n):
    if i % (n // 10) == 0:
        print(f"{i - offset} / {n}")
    try:
        res = generate_sequence(dataset["train"][i]["xmi"])
    except Exception as e:
        print(f"=== exception in {i}: {e}")
        with open("exception.xml", "w") as f:
            f.write(dataset["train"][i]["xmi"])
        print(dataset["train"][i]["code"])
        raise e


0 / 10000
exception: get_guard_name: <Element 'qualifier' at 0x7fd383e68f40> [('{http://www.w3.org/2001/XMLSchema-instance}type', 'java:PackageAccess'), ('package', '//@ownedElements.1')]
=== exception in 103729: get_guard_name: <Element 'qualifier' at 0x7fd383e68f40> [('{http://www.w3.org/2001/XMLSchema-instance}type', 'java:PackageAccess'), ('package', '//@ownedElements.1')]
private void addLocalRuntimeDefinition(
          final AvroLocalAppSubmissionParameters localAppSubmissionParams,
          final AvroJobSubmissionParameters jobSubmissionParameters,
          final MultiRuntimeDefinitionBuilder builder) {
    // create and serialize local configuration if defined
    final Configuration localModule = LocalDriverConfiguration.CONF
            .set(LocalDriverConfiguration.MAX_NUMBER_OF_EVALUATORS,
                    localAppSubmissionParams.getMaxNumberOfConcurrentEvaluators())
            // ROOT FOLDER will point to the current runtime directory
            .set(LocalDriverCo

Exception: get_guard_name: <Element 'qualifier' at 0x7fd383e68f40> [('{http://www.w3.org/2001/XMLSchema-instance}type', 'java:PackageAccess'), ('package', '//@ownedElements.1')]

In [10]:
idx = 103729

print(dataset["train"][idx]["originalLine"])
print(dataset["train"][idx]["code"])
with open("test_xmi.xml", "w") as f:
    f.write(dataset["train"][idx]["xmi"])

print("\n\n")

print(json.dumps(generate_sequence(dataset["train"][idx]["xmi"]), indent=2))


20808
private void addLocalRuntimeDefinition(
          final AvroLocalAppSubmissionParameters localAppSubmissionParams,
          final AvroJobSubmissionParameters jobSubmissionParameters,
          final MultiRuntimeDefinitionBuilder builder) {
    // create and serialize local configuration if defined
    final Configuration localModule = LocalDriverConfiguration.CONF
            .set(LocalDriverConfiguration.MAX_NUMBER_OF_EVALUATORS,
                    localAppSubmissionParams.getMaxNumberOfConcurrentEvaluators())
            // ROOT FOLDER will point to the current runtime directory
            .set(LocalDriverConfiguration.ROOT_FOLDER, ".")
            .set(LocalDriverConfiguration.JVM_HEAP_SLACK, 0.0)
            .set(LocalDriverConfiguration.CLIENT_REMOTE_IDENTIFIER, ClientRemoteIdentifier.NONE)
            .set(LocalDriverConfiguration.JOB_IDENTIFIER,
                    jobSubmissionParameters.getJobId().toString())
            .set(LocalDriverConfiguration.RUNTIME_NAMES,
  

In [146]:
with open("test_in.xml") as f:
    example = {"xmi": f.read()}
print(json.dumps(generate_sequence(example["xmi"]), indent=2))


[
  {
    "type": "newInstance",
    "new_type": "TestClass"
  },
  {
    "type": "scopedVariable",
    "name": "myA"
  },
  {
    "type": "methodInvocation",
    "to": [
      "myA"
    ],
    "method": "func1"
  },
  {
    "type": "methodInvocation",
    "to": [
      "myA"
    ],
    "method": "func2"
  },
  {
    "type": "blocks",
    "blocks": [
      {
        "name": "if",
        "guard": "1 == 2",
        "contents": [
          {
            "type": "methodInvocation",
            "to": [
              "myA"
            ],
            "method": "func3"
          }
        ]
      }
    ]
  },
  {
    "type": "blocks",
    "blocks": [
      {
        "name": "for init",
        "guard": null,
        "contents": [
          {
            "type": "scopedVariable",
            "name": "i"
          }
        ]
      },
      {
        "name": "for",
        "guard": "i < 5",
        "contents": [
          {
            "type": "methodInvocation",
            "to": [
           

In [166]:
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')

In [171]:
for i in range(100):
    example = dataset["train"][i]
    print(
        len(tokenizer(example["contents"])["input_ids"]),
        len(tokenizer(json.dumps(generate_sequence(example["xmi"])))["input_ids"]),
        len(tokenizer(json.dumps(generate_sequence(example["xmi"]), indent=2))["input_ids"]),
    )


141 201 334
85 43 71
84 47 83
69 43 71
71 70 123
68 43 71
68 47 83
53 25 44
322 577 991
134 24 37
150 24 37
167 24 37
66 69 119
48 25 40
151 218 364
120 68 109
149 194 312
74 109 193
59 78 128
135 219 377
65 82 128
78 91 143
77 117 205
74 84 146
100 96 171
76 110 188
104 149 247
78 144 239
119 67 114
126 67 114
112 108 171
175 243 421
198 350 619
117 191 364
53 22 35
150 208 360
64 51 83
220 188 305
121 158 261
74 46 76
53 50 76
62 68 115
114 124 222
81 97 169
135 133 233
76 96 154
145 313 508
120 187 302
72 123 203
84 42 70
85 71 124
121 134 207
68 91 157
81 260 402
233 408 686
152 280 474
149 208 369
90 148 255
89 148 255
400 764 1354
104 84 148
327 675 1114
143 281 467
84 109 172
180 326 535
85 95 154
168 171 287
186 287 473
59 47 79
72 66 105
84 105 183
104 3 3
102 73 116
182 295 514
117 177 318
89 105 184
97 108 185
129 253 446
108 152 265
72 24 37
72 24 37
71 70 119
105 65 106
76 71 120
215 345 569
184 336 548
83 95 168
89 118 209
109 139 246
184 255 433
181 321 572
61 30 51
60 8